In [1]:
import carla, time, pygame, cv2, math
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()
carla_settings = world.get_settings()

In [3]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_vehicle(world, vehicle_index=0, spawn_index=0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle


def spawn_pedestrian(spawn_index=0):
    blueprint_library = world.get_blueprint_library()
    pedestrian_bp = blueprint_library.filter('walker.pedestrian.*')[0]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    pedestrian = world.spawn_actor(pedestrian_bp, spawn_point)
    return pedestrian

def callback(image):
    image.convert(carla.ColorConverter.CityScapesPalette)
    time.sleep(3)
    image.save_to_disk('output/%.6d.png' % image.frame)

def move_spectator_to(transform, spectator, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

In [9]:
throttle = 0.6

def move_forward(vehicle, duration):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)
    start_time = time.time()
    while time.time() - start_time < duration:
        world.tick()
        time.sleep(0.1)

def steer_left(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    control.steer = -0.38
    vehicle.apply_control(control)
    world.tick()
    time.sleep(2)

def steer_right(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    control.steer = 0.38
    vehicle.apply_control(control)
    world.tick()
    time.sleep(2)

In [17]:
pygame.init()
# focus on the pygame window to get the key pressed
pygame.display.set_mode((400, 300))

vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

original_view = 0
is_reversed = False

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            print(f"Key pressed: {pygame.key.name(key)}")
            if key == pygame.QUIT or key == pygame.K_ESCAPE:
                running = False
            elif key ==pygame.K_r: # reverse
                is_reversed = not is_reversed
    move_spectator_to(vehicle.get_transform(), spectator, yaw = 180 if is_reversed else original_view)
    world.tick()
    pygame.display.flip()

pygame.quit()

Key pressed: r
Key pressed: r
Key pressed: escape


In [14]:
vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)
vehicle.set_autopilot(True)

spectator_transform = vehicle.get_transform()
move_spectator_to(spectator_transform, spectator)

camera.listen(callback)

for i in range(10):
    pedestrian = spawn_pedestrian(spawn_index=i)

AttributeError: 'Walker' object has no attribute 'set_autopilot'

In [15]:
if vehicle:
    vehicle.destroy()
if camera:
    camera.stop()
    camera.destroy()
for actor in world.get_actors().filter('walker.pedestrian.*'):
    actor.destroy()

: 

In [4]:
pygame.init()
screen = pygame.display.set_mode((400, 300))

running = True
moving_forward = False

# target_vehicle = spawn_vehicle()
target_pedestrian = spawn_pedestrian()

time.sleep(1)

target_transform = target_pedestrian.get_transform()
target_transform.location += carla.Location(x=35, y=-1)
target_pedestrian.set_transform(target_transform)

# controller_bp = world.get_blueprint_library().find('controller.ai.walker')
# controller = world.spawn_actor(controller_bp, target_pedestrian.get_transform(), target_pedestrian)


time.sleep(1)

ego_vehicle = spawn_vehicle(world)

rad_bp = world.get_blueprint_library().find('sensor.other.radar')
rad_bp.set_attribute('horizontal_fov', str(35))
rad_bp.set_attribute('vertical_fov', str(20))
rad_bp.set_attribute('range', str(20))
rad_location = carla.Location(x=2.0, z=1.0)
rad_rotation = carla.Rotation(pitch=5)
rad_transform = carla.Transform(rad_location,rad_rotation)
rad_ego = world.spawn_actor(rad_bp,rad_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)

def rad_callback(radar_data):
    global moving_forward

    # min_distance_to_brake = 10.0
    # brake_strength = 0.5
    relative_velocity = 0

    velocity_range = 7.5 # m/s
    current_rot = radar_data.transform.rotation
    for detect in radar_data:
        azi = math.degrees(detect.azimuth)
        alt = math.degrees(detect.altitude)
        velocity = detect.velocity
        depth = detect.depth

        ego_velocity = ego_vehicle.get_velocity() 
        ego_speed = math.sqrt(ego_velocity.x ** 2 + ego_velocity.y ** 2)
        relative_velocity = ego_speed - relative_velocity
        
        if relative_velocity and relative_velocity != 0:
            ttc = depth / relative_velocity
            if ttc > 0:
                print(f"Time to Collision (TTC): {ttc:.2f} seconds")

                if ttc < 2.0:
                     print("Warning! Braking required.")
                     moving_forward = False

        # The 0.25 adjusts a bit the distance so the dots can
        # be properly seen
        fw_vec = carla.Vector3D(x=detect.depth - 0.25)
        carla.Transform(
            carla.Location(),
            carla.Rotation(
                pitch=current_rot.pitch + alt,
                yaw=current_rot.yaw + azi,
                roll=current_rot.roll)).transform(fw_vec)

        def clamp(min_v, max_v, value):
            return max(min_v, min(value, max_v))

        norm_velocity = detect.velocity / velocity_range # range [-1, 1]
        r = int(clamp(0.0, 1.0, 1.0 - norm_velocity) * 255.0)
        g = int(clamp(0.0, 1.0, 1.0 - abs(norm_velocity)) * 255.0)
        b = int(abs(clamp(- 1.0, 0.0, - 1.0 - norm_velocity)) * 255.0)
        world.debug.draw_point(
            radar_data.transform.location + fw_vec,
            size=0.075,
            life_time=0.06,
            persistent_lines=False,
            color=carla.Color(r, g, b))
        
rad_ego.listen(lambda radar_data: rad_callback(radar_data))

while running:
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            if key == pygame.QUIT or key == pygame.K_ESCAPE:
                running = False
            elif key == pygame.K_w:
                moving_forward = True
                # controller.start()
                # next_location = target_pedestrian.get_transform()
                # # next_location.location += carla.Location(y=10)
                # controller.go_to_location(carla.Location(x=next_location.location.x, y=10))
            elif key == pygame.K_s:
                moving_forward = False
            elif key == pygame.K_p:
                target_pedestrian.apply_control(carla.WalkerControl(destination=carla.Vector3D(y=10), speed = 1.0, jump=False))

    if moving_forward:
        control = carla.VehicleControl(throttle=0.8)
    else:
        control = carla.VehicleControl(throttle=0.0)

    ego_vehicle.apply_control(control)
    world.tick()
    pygame.display.flip()

pygame.quit()
rad_ego.destroy()
ego_vehicle.destroy()
for actor in world.get_actors().filter('walker.pedestrian.*'):
    actor.destroy()

Time to Collision (TTC): 5410.84 seconds
Time to Collision (TTC): 4121.39 seconds
Time to Collision (TTC): 6330.30 seconds
Time to Collision (TTC): 9381.71 seconds
Time to Collision (TTC): 769.49 seconds
Time to Collision (TTC): 1037.75 seconds
Time to Collision (TTC): 1880.60 seconds
Time to Collision (TTC): 71247.18 seconds
Time to Collision (TTC): 57236.75 seconds
Time to Collision (TTC): 4588.39 seconds
Time to Collision (TTC): 5683.64 seconds
Time to Collision (TTC): 3664.96 seconds
Time to Collision (TTC): 4326.50 seconds
Time to Collision (TTC): 3781.98 seconds
Time to Collision (TTC): 4804.14 seconds
Time to Collision (TTC): 5386.65 seconds
Time to Collision (TTC): 8573.48 seconds
Time to Collision (TTC): 9491.06 seconds
Time to Collision (TTC): 12766.13 seconds
Time to Collision (TTC): 13638.38 seconds
Time to Collision (TTC): 14832.52 seconds
Time to Collision (TTC): 17996.44 seconds
Time to Collision (TTC): 16674.85 seconds
Time to Collision (TTC): 30462.39 seconds
Time to C

ArgumentError: Python argument types in
    WalkerControl.__init__(WalkerControl)
did not match C++ signature:
    __init__(struct _object * __ptr64, class carla::geom::Vector3D direction=<carla.libcarla.Vector3D object at 0x000002467E01A7B0>, float speed=0.0, bool jump=False)
    __init__(struct _object * __ptr64)

: 

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

: 